# Flugpreis Vorhersage - Kaufen oder Warten?
## Projektarbeit Data Mining
___
### Wintersemester 2021/22
### Gruppe G:
Max Grundmann - s0559326
### Inhalte
1. Problemanalyse
2. Explorative Datenanalyse
3. Weitere Features
4. Praktische Überlegungen
___

#### Benötigte Bibliotheken laden

In [27]:
from random import sample
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier

#### Vorbereitete Daten laden

In [4]:
n = 40

dirname = os.getcwd()
filename = os.path.join(dirname, f'../Data/prepped/train_set_n{n}.csv')

try:
    data = pd.read_csv(filename, index_col=0)
except FileNotFoundError as e:
    print('Datei konnte nicht gefunden werden.')

Daten in Trainings- und Testdaten aufteilen.
80% Trainingsdaten, 20% Testdaten
Stratify = Yes stellt außerdem sicher, dass das Verhältnis der Klassen in beiden Datensätzen gleich ist.

In [5]:
# Daten in Trainings- und Testdaten aufteilen
y = data['buy']
X = data.drop(columns=['buy'], axis=1)

X.columns = X.columns.map(str)


for i in range(n):
    X[str(i)] = np.where(X[str(i)].isnull(), X.Price_In_Eur, X[str(i)])

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99, stratify=y)

#### Auswahl eines ML-Verfahrens

Folgenden Methode dient dem Automatisierten Training und Evaluierung eines ML-Modells. Ziel ist es, eine Baseline für mehrere unterschiedliche Verfahren zu etablieren. Das beste Modell wird anschließend über Hyperparameter Tuning weiter optimiert.

In [6]:
def evaluate_model(model, x_train, y_train, x_test, y_test):
    model = model.fit(x_train, y_train)
    print('Training Score:', model.score(x_train, y_train))
    
    predicted = pd.DataFrame(model.predict(x_test))
    
    probs = pd.DataFrame(model.predict_proba(x_test))
    print('Test Score:', metrics.accuracy_score(y_test, predicted))
    
    print(metrics.classification_report(y_test, predicted))
    return model

Die Support Vector Machine wurde im weiteren Vorgehen nicht berücksichtigt, da die Trainingszeiten unverhältnismäßig lag dauern. Das dürfte darin begründet liegen, dass die Trainingskomplexität einer SVM bei O(n^2) liegt und entsprechend stark bzw. schlecht mit der Anzahl an Trainingsdaten skaliert. Entsprechend limitiert vermutlich auch mein Arbeitsspeicher die Berechnung.

In [28]:
list_of_models = [RandomForestClassifier(), 
                # SVC(kernel='linear'), 
                  XGBClassifier(),
                  AdaBoostClassifier(),
                  GradientBoostingClassifier(),
                  tree.DecisionTreeClassifier(max_depth=10), 
                  LogisticRegression(), 
                  BaggingClassifier(),
                  KNeighborsClassifier(n_neighbors=5), 
                  GaussianNB()]

for model in list_of_models:
    print(model)
    evaluate_model(model, x_train, y_train, x_test, y_test)
    print('-------------------------------------------------------------------------------')

RandomForestClassifier()
Training Score: 0.9999103125607258
Test Score: 0.9203587443946188
              precision    recall  f1-score   support

           0       0.92      0.98      0.95     12981
           1       0.91      0.71      0.80      3744

    accuracy                           0.92     16725
   macro avg       0.92      0.85      0.88     16725
weighted avg       0.92      0.92      0.92     16725

-------------------------------------------------------------------------------
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=N

C:\Users\Max\Anaconda3\envs\py39\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:42:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Training Score: 0.9379960836484851
Test Score: 0.89745889387145
              precision    recall  f1-score   support

           0       0.90      0.97      0.94     12981
           1       0.86      0.65      0.74      3744

    accuracy                           0.90     16725
   macro avg       0.88      0.81      0.84     16725
weighted avg       0.90      0.90      0.89     16725

-------------------------------------------------------------------------------
AdaBoostClassifier()
Training Score: 0.8285774077340469
Test Score: 0.8208071748878923
              precision    recall  f1-score   support

           0       0.84      0.96      0.89     12981
           1       0.70

C:\Users\Max\Anaconda3\envs\py39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training Score: 0.8154381978803869
Test Score: 0.8089686098654708
              precision    recall  f1-score   support

           0       0.82      0.96      0.89     12981
           1       0.67      0.28      0.40      3744

    accuracy                           0.81     16725
   macro avg       0.75      0.62      0.64     16725
weighted avg       0.79      0.81      0.78     16725

-------------------------------------------------------------------------------
BaggingClassifier()
Training Score: 0.995156878279197
Test Score: 0.91898355754858
              precision    recall  f1-score   support

           0       0.92      0.98      0.95     12981
           1       0.90      0.71      0.80      3744

    accuracy                           0.92     16725
   macro avg       0.91      0.85      0.87     16725
weighted avg       0.92      0.92      0.92     16725

-------------------------------------------------------------------------------
KNeighborsClassifier()


MemoryError: Unable to allocate 1.00 GiB for an array with shape (2006, 66899) and data type float64

### Hyperparameter Tuning eines Random Forest Classifiers

Im einem Vergleich ohne jegliches Hyperparameter Tuning hat der Random Forest Classifier für den Datensatz die höchste Test-Genauigkeit erzielt. Allerdings hat sich bereits dort gezeigt, dass der Classifier ohne weiteres Tuning stark zum overfitting neigt, was an einer Trainingsgenauigkeit von 99% deutlich wird. Für das weitere Tuning wird daher eine K-Fold Cross-Validation mit K=5 durchgeführt.

In [39]:
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std

In [11]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

#### Cross-Validation Baseline

In [19]:
def performCrossValidation(model, x_train, y_train, splits=10):
    print(f'{model} mit splits={splits}')
    # cv = KFold(n_splits=splits, random_state=199, shuffle=True)
    cv = RepeatedStratifiedKFold(n_splits=splits, n_repeats=3, random_state=1)
    return cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

In [ ]:
display_scores(performCrossValidation(RandomForestClassifier(), x_train, y_train))

In [ ]:
display_scores(performCrossValidation(XGBClassifier(), x_train, y_train))

In [23]:
display_scores(performCrossValidation(AdaBoostClassifier(), x_train, y_train, 5))

AdaBoostClassifier()
Scores: [0.82675635 0.82929746 0.82331839 0.83213752 0.82988265]
Mean: 0.8282784735822408
Standard deviation: 0.00301407605651641


In [26]:
display_scores(performCrossValidation(GradientBoostingClassifier(), x_train, y_train, 2))

GradientBoostingClassifier()
Scores: [0.83751868 0.84265598]
Mean: 0.8400873341701605
Standard deviation: 0.002568649566274117


#### Grid Search für Random Forrest Classfier

In [31]:
from sklearn.model_selection import RandomizedSearchCV

In [37]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# rf_random = RandomizedSearchCV(estimator = rf, 
#                                param_distributions = random_grid, 
#                                n_iter = 100, 
#                                cv = 3, 
#                                verbose=2, 
#                                random_state=42, 
#                                n_jobs = -1)

# rf_random.fit(x_train, y_train)
# rf_random.best_params_

In [43]:
random_grid = {'n_estimators': [50, 200, 400],
               'max_features': ['auto', 0.25, 0.5, 1.0],
               'max_depth': [4, 8,16],
               'min_samples_split': [2, 5, 10],
            #    'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=2, random_state=1)

rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 20, 
                               cv = 5, 
                               verbose=10, 
                               random_state=42, 
                               scoring='accuracy',
                               n_jobs = -1)

rf_random.fit(x_train, y_train)
rf_random.best_params_

Fitting 5 folds for each of 20 candidates, totalling 100 fits


{'n_estimators': 50,
 'min_samples_split': 5,
 'max_features': 0.5,
 'max_depth': 16,
 'bootstrap': False}

In [46]:
evaluate_model(RandomForestClassifier(
    n_estimators= 50,
    min_samples_split= 5,
    # min_samples_leaf= 1,
    max_features= 'auto',
    max_depth= 10,
    bootstrap= False), x_train, y_train, x_test, y_test)

Training Score: 0.8431815124291843
Test Score: 0.8289387144992526
              precision    recall  f1-score   support

           0       0.82      0.99      0.90     12981
           1       0.92      0.26      0.40      3744

    accuracy                           0.83     16725
   macro avg       0.87      0.63      0.65     16725
weighted avg       0.84      0.83      0.79     16725



RandomForestClassifier(bootstrap=False, max_depth=10, min_samples_split=5,
                       n_estimators=50)

In [44]:
evaluate_model(RandomForestClassifier(
    n_estimators= 50,
    min_samples_split= 5,
    # min_samples_leaf= 1,
    max_features= 0.5,
    max_depth= 16,
    bootstrap= False), x_train, y_train, x_test, y_test)

Training Score: 0.9770549634523685
Test Score: 0.9162929745889388
              precision    recall  f1-score   support

           0       0.92      0.98      0.95     12981
           1       0.92      0.69      0.79      3744

    accuracy                           0.92     16725
   macro avg       0.92      0.83      0.87     16725
weighted avg       0.92      0.92      0.91     16725



RandomForestClassifier(bootstrap=False, max_depth=16, max_features=0.5,
                       min_samples_split=5, n_estimators=50)

#### Cross Validation mit Hyperparameter Tuning

In [ ]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)

model = RandomForestClassifier(n_estimators= 200, 
                               max_features= 'sqrt')

scores = cross_val_score(model, x_train.drop(['request_is_weekend', 
                                              'request_month_sin', 
                                              'request_month_cos'],1), y_train, scoring='accuracy', cv=cv, n_jobs=-1)

model = model.fit(x_train, y_train)

In [ ]:
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

# Predictions on the test dataset
predicted = pd.DataFrame(model.predict(x_test))
# Probabilities on the test dataset
probs = pd.DataFrame(model.predict_proba(x_test))
print('Test Score:', metrics.accuracy_score(y_test, predicted))

print(metrics.classification_report(y_test, predicted))
print(model.feature_importances_)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
figure(figsize=(10, 10), dpi=80)

sorted_idx = model.feature_importances_.argsort()
plt.barh(x_train.columns[sorted_idx], model.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance")

___

Konfidenzintervall berechnen? Sinnvoll für Classification?

## 3. Neural Network

In [46]:
from tensorflow import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [48]:
n_cols = x_train.shape[1]

# Set up the model: model
model = Sequential()

# Add the first layer
model.add(Dense(512, activation='relu', input_shape=(n_cols,)))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

opt = keras.optimizers.SGD(learning_rate=0.1)

model.compile(
    optimizer='sgd', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

# early_stopping_monitor = EarlyStopping(patience=3)

model.fit(
    X, 
    y, 
    validation_split=0.2,
    batch 
    epochs=200 
    # callbacks=[early_stopping_monitor]
    )

Epoch 1/200
2091/2091 [==============================] - 6s 2ms/step - loss: 0.5252 - accuracy: 0.7592 - val_loss: 0.3912 - val_accuracy: 0.8380
Epoch 2/200
2091/2091 [==============================] - 5s 2ms/step - loss: 0.4859 - accuracy: 0.7803 - val_loss: 0.3771 - val_accuracy: 0.8468
Epoch 3/200
2091/2091 [==============================] - 5s 2ms/step - loss: 0.4673 - accuracy: 0.7925 - val_loss: 0.3704 - val_accuracy: 0.8504
Epoch 4/200
2091/2091 [==============================] - 5s 2ms/step - loss: 0.4553 - accuracy: 0.7980 - val_loss: 0.3692 - val_accuracy: 0.8505
Epoch 5/200
2091/2091 [==============================] - 5s 2ms/step - loss: 0.4476 - accuracy: 0.7992 - val_loss: 0.3663 - val_accuracy: 0.8529
Epoch 6/200
2091/2091 [==============================] - 5s 2ms/step - loss: 0.4415 - accuracy: 0.8022 - val_loss: 0.3750 - val_accuracy: 0.8529
Epoch 7/200
2091/2091 [==============================] - 5s 2ms/step - loss: 0.4337 - accuracy: 0.8059 - val_loss: 0.3709 - val_ac

In [56]:
import matplotlib.pyplot as plt

plt.plot(model.history['val_loss'], 'r')
plt.xlabel('Epochs')
plt.ylabel('Validation score')
plt.show()

TypeError: 'History' object is not subscriptable

In [40]:
pred = model.predict(x_test)

In [43]:
print(metrics.classification_report(y_test, pred))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## 3. Feature Engineering

Für eine optimale Vorhersage wären natürlich Informationen zur Anzahl tatsächlich verfügbarer Sitzplätze oder detailiertere Informationen zu verkauften Ticket-Typen (Business Class, Economy Class, Geschäftsreisende, etc. - wenn auch vermutlich auf kürzeren Strecken wie denen im Datensatz weniger relevant) hilfreich, die jedoch typischerweise nicht öffentlich verfügbar sind (vgl. Manolis Papadakis 2021: Predicting Airfare Prices, S. 1. http://cs229.stanford.edu/proj2012/Papadakis-PredictingAirfarePrices.pdf Letzter Zugriff: 06.12.2021).

In [ ]:
data.skew()

Die Verteilung der Features (mit Ausnahme der Preisbasierten Spalten) zeigt mit Werten zwischen -0,5 und 0,5 keine Hinweise auf deutlich asymetrische Verteilungen (vgl. https://medium.com/@atanudan/kurtosis-skew-function-in-pandas-aa63d72e20de) . Grundsätzlich besteht bei den vorhandenen Features keine Notwendigkeit zur Normalisierung. Nur kategorische Werte müssen für die Verwendung in den Modellen noch codiert werden.

Über die bestehenden Features hinaus werden folgende Features für die Vorhersage ergänzt:

- Wochentag
- Tage bis zum Flug
- Tage bis zum nächsten Feiertag

Ob diese Features einen tatsächlichen Mehrwert für die Vorhersage liefern, muss im zweiten Teil der Projektarbeit ermittelt werden.